In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc 
import numpy as np 
import pandas as pd 
import os, sys 
sys.path.append('../../src')

In [3]:
from spaceoracle.tools.network import MouseSpleenRegulatoryNetwork
co_grn = MouseSpleenRegulatoryNetwork()

2024-11-25 20:04:56.108740: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data/slideseq_spleen.h5ad')
adata

AnnData object with n_obs × n_vars = 6891 × 3000
    obs: 'n_genes', 'n_UMIs', 'Cell_Type', 'cell_type_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'log10_n_UMIs', 'log10_n_genes', 'donor_id', 'is_primary_data', 'suspension_type', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'clusters'
    var: 'gene', 'n_beads', 'n_UMIs', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'fea

In [5]:
from spaceoracle.oracles import SpaceTravLR

star = SpaceTravLR(
    adata, 
    save_dir='/ix/djishnu/shared/djishnu_kor11/models_spleen',
    annot='clusters',
    grn=co_grn
)

star.adata

Calculating MAGIC...
  Running MAGIC on 6891 cells and 3000 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 1.38 seconds.
    Calculating KNN search...
    Calculated KNN search in 4.52 seconds.
    Calculating affinities...
    Calculated affinities in 4.48 seconds.
  Calculated graph and diffusion operator in 10.40 seconds.
  Calculating imputation...
  Calculated imputation in 1.14 seconds.
Calculated MAGIC in 11.57 seconds.


AnnData object with n_obs × n_vars = 6891 × 3000
    obs: 'clusters'
    obsm: 'spatial'
    layers: 'normalized_count', 'raw_count', 'imputed_count'

In [15]:
from spaceoracle.models.parallel_estimators import SpatialCellularProgramsEstimator

# test run to make sure everything is working

estimator = SpatialCellularProgramsEstimator(
    adata=star.adata,
    target_gene='E2f2',
    layer='imputed_count',
    cluster_annot='clusters', 
    grn=co_grn
)

estimator.fit(num_epochs=200, threshold_lambda=1e-12, learning_rate=5e-4)


Fitting E2f2 with 148 modulators
	84 Transcription Factors
	16 Ligand-Receptor Pairs
	48 TranscriptionFactor-Ligand Pairs
0: 0.7570 | 0.7196
1: 0.8433 | 0.8334
2: 0.6976 | 0.6645
3: 0.9805 | 0.9380
4: 0.9509 | 0.9428


see train.py and submit with run.sh

In [18]:
adata.layers['imputed_count'] = star.adata.layers['imputed_count']
adata.write_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data/slideseq_spleen.h5ad')